In [1]:
import  numpy as np
import  matplotlib.pyplot as plt
import cv2
import pandas as pd
import requests
import cv2
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.preprocessing import normalize

C:\Users\lenovo\AppData\Local\Temp\ipykernel_12088\4283929640.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the pre-trained ResNet model
resnet = models.resnet50(pretrained=True)

# Remove the last layer
feature_extractor = torch.nn.Sequential(*(list(resnet.children())[:-1]))


C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
data=pd.read_csv('A2_Data.csv')
def preprocess_image_urls(image_column):
    image_column = image_column.apply(lambda x: x.replace("[", "").replace("]", "").replace("'", "").split(", "))
    return image_column

# Preprocess the 'Image' column
data['Image'] = preprocess_image_urls(data['Image'])
data.head()

,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,Loving these vintage springs on my vintage str...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."
1,1205,[https://images-na.ssl-images-amazon.com/image...,Works great as a guitar bench mat. Not rugged ...,"[array([[0.38690877, 0.5909363 , 0.47527072, ...."
2,1708,[https://images-na.ssl-images-amazon.com/image...,We use these for everything from our acoustic ...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."
3,2078,[https://images-na.ssl-images-amazon.com/image...,Great price and good quality. It didn't quite...,"[array([[0.38690877, 0.5909363 , 0.47527072, ...."
4,801,[https://images-na.ssl-images-amazon.com/image...,I bought this bass to split time as my primary...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."


In [49]:
image=None

def adjust_brightness_and_contrast(image, brightness=0, contrast=0):
        # Alpha controls contrast; Beta controls brightness.
        alpha = 1 + contrast / 127
        beta = brightness
        adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
        return adjusted_image

def preprocess(image_url):
    response = requests.get(image_url)
    image = cv2.imdecode(np.frombuffer(response.content, np.uint8), -1)
    if image is None:
        return None
    image_height, image_width = image.shape[:2]


    #border removal
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
    contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    x,y,w,h = cv2.boundingRect(contours[0])
    image = image[y:y+h,x:x+w]
    image = cv2.resize(image, (80, 80))


    # Random flip
    # flip_direction = np.random.choice(["horizontal", "vertical"])
    # if flip_direction == "horizontal":
    #     image = cv2.flip(image, 1)  # 1: Flip horizontally
    # elif flip_direction == "vertical":
    #     image = cv2.flip(image, 0)  # 0: Flip vertically

    

    #pixel normalisation
    def normalize_image(image):
        normalized_image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        return normalized_image

    image = normalize_image(image)
    

    #brightness and contrast adjustment
    def calculate_brightness_and_contrast(image):
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        mean, std_dev = cv2.meanStdDev(gray_image)
        return mean[0][0], std_dev[0][0]

    

    mean_brightness, std_dev_contrast = calculate_brightness_and_contrast(image)

    # Set a threshold for deciding whether to adjust brightness and contrast
    brightness_threshold = 20  
    contrast_threshold = 10 

    # Adjust brightness and contrast only if needed
    if mean_brightness < brightness_threshold or std_dev_contrast < contrast_threshold:
        brightness = 20  
        contrast = 20  
        adjusted_image = adjust_brightness_and_contrast(image, brightness=brightness, contrast=contrast)
    else:
        adjusted_image = image

    return image


image_url = data['Image'][5][0]
image = preprocess(image_url)


# plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# plt.axis('off')
# plt.show()

In [50]:



def extract_features(image):
    img = cv2.resize(image, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_tensor = transforms.ToTensor()(img)
    # normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # img_normalized = normalize(img_tensor)
    # img_normalized = img_normalized.unsqueeze(0)
    with torch.no_grad():
        features = feature_extractor(img_tensor.unsqueeze(0))
    features = features.view(features.size(0), -1)
    features = features.numpy()
    return features

extracted_features = extract_features(image)


normalized_features = normalize(extracted_features, axis=1, norm='l2')
print(len(normalized_features[0]))
print(normalized_features[0])



2048
[0.01630095 0.02581563 0.02321972 ... 0.01390954 0.02334862 0.01602637]


In [51]:
def imageTOvec(image_id):
    image_url = image_id
    # print(image_url)
    image = preprocess(image_url)
    if image is None:
        return None
    features = extract_features(image)
    return features

def cosine_similarity(image_id1, image_id2):
    features1 = imageTOvec(image_id1)
    features2 = imageTOvec(image_id2)
    similarity = np.dot(features1, features2.T)
    return similarity


In [52]:
print(imageTOvec(data['Image'][200][0]))
print(imageTOvec(data['Image'][201][0]))
print(data['Image'][200][0])
print(data['Image'][201][0])

print(imageTOvec(data['Image'][202][0]))
print(imageTOvec(data['Image'][2][0]))
print(cosine_similarity(data['Image'][200][0],data['Image'][202][0]))


[[0.373968   0.52164257 0.4784318  ... 0.34860444 0.5204003  0.32329345]]
[[0.41571122 0.5506941  0.51992995 ... 0.38357964 0.46930087 0.37079674]]
https://images-na.ssl-images-amazon.com/images/I/71LPADvZmeL._SY88.jpg
https://images-na.ssl-images-amazon.com/images/I/71-ReuliVwL._SY88.jpg
[[0.35981348 0.5382129  0.519022   ... 0.30876225 0.45836905 0.2773529 ]]
[[0.34926912 0.5903471  0.48663428 ... 0.31742734 0.4899901  0.32530886]]
[[451.6257]]


In [48]:
# print(extract_features(preprocess(data['Image'][200][0])))
print(preprocess(data['Image'][201][0][0]))

# print(extract_features(preprocess(data['Image'][201][0])))

[[[0.14509805 0.1254902  0.13333334]
  [0.21960786 0.20000002 0.20784315]
  [0.12941177 0.10980393 0.12156864]
  ...
  [0.05490196 0.05490196 0.10196079]
  [0.04705883 0.04313726 0.08235294]
  [0.02745098 0.02352941 0.0627451 ]]

 [[0.15294118 0.13333334 0.14117648]
  [0.23137257 0.21176472 0.21960786]
  [0.1764706  0.15686275 0.17254902]
  ...
  [0.0509804  0.05490196 0.10588236]
  [0.09411766 0.09019608 0.12941177]
  [0.09019608 0.08627451 0.12941177]]

 [[0.20000002 0.18039216 0.18823531]
  [0.2784314  0.25882354 0.26666668]
  [0.15686275 0.13725491 0.14901961]
  ...
  [0.03137255 0.03137255 0.08627451]
  [0.0509804  0.05490196 0.09803922]
  [0.07843138 0.07843138 0.1254902 ]]

 ...

 [[0.2784314  0.24705884 0.2901961 ]
  [0.2784314  0.24705884 0.2901961 ]
  [0.21960786 0.18823531 0.23137257]
  ...
  [0.21960786 0.18823531 0.22352943]
  [0.20784315 0.1764706  0.21960786]
  [0.17254902 0.14117648 0.18431373]]

 [[0.10980393 0.07843138 0.12156864]
  [0.3529412  0.32156864 0.3647059 ]


In [47]:
print("aaaa")
print(preprocess(data['Image'][200][0][0]))

aaaa
[[[0.14509805 0.1254902  0.13333334]
  [0.21960786 0.20000002 0.20784315]
  [0.12941177 0.10980393 0.12156864]
  ...
  [0.05490196 0.05490196 0.10196079]
  [0.04705883 0.04313726 0.08235294]
  [0.02745098 0.02352941 0.0627451 ]]

 [[0.15294118 0.13333334 0.14117648]
  [0.23137257 0.21176472 0.21960786]
  [0.1764706  0.15686275 0.17254902]
  ...
  [0.0509804  0.05490196 0.10588236]
  [0.09411766 0.09019608 0.12941177]
  [0.09019608 0.08627451 0.12941177]]

 [[0.20000002 0.18039216 0.18823531]
  [0.2784314  0.25882354 0.26666668]
  [0.15686275 0.13725491 0.14901961]
  ...
  [0.03137255 0.03137255 0.08627451]
  [0.0509804  0.05490196 0.09803922]
  [0.07843138 0.07843138 0.1254902 ]]

 ...

 [[0.2784314  0.24705884 0.2901961 ]
  [0.2784314  0.24705884 0.2901961 ]
  [0.21960786 0.18823531 0.23137257]
  ...
  [0.21960786 0.18823531 0.22352943]
  [0.20784315 0.1764706  0.21960786]
  [0.17254902 0.14117648 0.18431373]]

 [[0.10980393 0.07843138 0.12156864]
  [0.3529412  0.32156864 0.36470

In [32]:
data['Image_Vectors'] = None
# iterate for first 1/3rd of dataset
for i, row in data.iterrows():
    
    # print(i)""
    vectors = []
    for url in row['Image']:
        vector = imageTOvec(url)
        if vector is not None:
            vectors.append(vector)
    if len(vectors) ==0:
        # drop the whole row
        data = data.drop(i)
    else:
        data.at[i, 'Image_Vectors'] = vectors
    

# Print the updated DataFrame
print(data)

ConnectionError: HTTPSConnectionPool(host='images-na.ssl-images-amazon.com', port=443): Max retries exceeded with url: /images/I/71LPADvZmeL._SY88.jpg (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000017B25549090>: Failed to resolve 'images-na.ssl-images-amazon.com' ([Errno 11001] getaddrinfo failed)"))

In [7]:
data.head()

,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,Loving these vintage springs on my vintage str...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."
1,1205,[https://images-na.ssl-images-amazon.com/image...,Works great as a guitar bench mat. Not rugged ...,"[array([[0.38690877, 0.5909363 , 0.47527072, ...."
2,1708,[https://images-na.ssl-images-amazon.com/image...,We use these for everything from our acoustic ...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."
3,2078,[https://images-na.ssl-images-amazon.com/image...,Great price and good quality. It didn't quite...,"[array([[0.38690877, 0.5909363 , 0.47527072, ...."
4,801,[https://images-na.ssl-images-amazon.com/image...,I bought this bass to split time as my primary...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."


In [8]:
droped_data = data.dropna()

In [19]:
#  drop rows with review text being nan
print(droped_data)

     Unnamed: 0                                              Image  \
0          3452  [https://images-na.ssl-images-amazon.com/image...   
1          1205  [https://images-na.ssl-images-amazon.com/image...   
2          1708  [https://images-na.ssl-images-amazon.com/image...   
3          2078  [https://images-na.ssl-images-amazon.com/image...   
4           801  [https://images-na.ssl-images-amazon.com/image...   
..          ...                                                ...   
995        1265  [https://images-na.ssl-images-amazon.com/image...   
996        1882  [https://images-na.ssl-images-amazon.com/image...   
997        1547  [https://images-na.ssl-images-amazon.com/image...   
998        1004  [https://images-na.ssl-images-amazon.com/image...   
999        1306  [https://images-na.ssl-images-amazon.com/image...   

                                           Review Text  \
0    Loving these vintage springs on my vintage str...   
1    Works great as a guitar bench mat. Not

In [9]:
#  preprorcess revierw texts
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer as ps

# Download the stopwords corpus if you haven't already
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

def preprocess_review_text(review_text):
    # print(review_text)
    # print index of the reivew text
    review_text = str(review_text)
    review_text = review_text.lower()
    review_text = re.sub(r'[^\w\s]', '', review_text)
    # remove stop words
    review_text = ' '.join([word for word in review_text.split() if word not in stop_words])
    # do stemming
    review_text = ' '.join([ps().stem(word) for word in review_text.split()])
    return review_text



data['Review Text'] = data['Review Text'].apply(preprocess_review_text)

data.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,love vintag spring vintag strat good tension g...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."
1,1205,[https://images-na.ssl-images-amazon.com/image...,work great guitar bench mat rug enough abus ta...,"[array([[0.38690877, 0.5909363 , 0.47527072, ...."
2,1708,[https://images-na.ssl-images-amazon.com/image...,use everyth acoust bass ukulel know smaller mo...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."
3,2078,[https://images-na.ssl-images-amazon.com/image...,great price good qualiti didnt quit match radi...,"[array([[0.38690877, 0.5909363 , 0.47527072, ...."
4,801,[https://images-na.ssl-images-amazon.com/image...,bought bass split time primari bass dean edg m...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."


In [5]:
import numpy as np
from collections import Counter
import math

def tokenize(text):
    # Split text into tokens (words)
    return text.split()

def calculate_tf(text):
    # Calculate term frequency (TF) for each word in the text
    tokens = tokenize(text)
    word_count = Counter(tokens)
    total_words = len(tokens)
    tf = {word: count / total_words for word, count in word_count.items()}
    return tf

def calculate_idf(documents):
    # Calculate inverse document frequency (IDF) for each word
    total_documents = len(documents)
    all_words = set([word for document in documents for word in tokenize(document)])
    idf = {}
    for word in all_words:
        doc_count = sum([1 for document in documents if word in tokenize(document)])
        idf[word] = math.log10(total_documents / doc_count)
    return idf

def calculate_tfidf(text, idf):
    # Calculate TF-IDF for each word in the text using precomputed IDF values
    tf = calculate_tf(text)
    tfidf = {word: tf[word] * idf[word] for word in tf}
    return tfidf

# make tfidf form review text
dataset = data['Review Text'].tolist()

# Calculate IDF for the dataset
idf = calculate_idf(dataset)

# Calculate TF-IDF for each document
tfidf_matrix = []
for document in dataset:
    tfidf = calculate_tfidf(document, idf)
    tfidf_matrix.append(tfidf)



In [22]:
#  normalise the tfidf matrix

# tfidf_matrix = normalize(tfidf_matrix, axis=1, norm='l2')



#  print tfidk in a tablular manner
tfidf_df = pd.DataFrame(tfidf_matrix)
#  convert thge nan to zeros
tfidf_df = tfidf_df.fillna(0)
#  print the first 5 rows
print(tfidf_df.head())



       love   vintag   spring     strat      good   tension     great  \
0  0.060206  0.23263  0.28732  0.091102  0.045883  0.139794  0.032990   
1  0.000000  0.00000  0.00000  0.000000  0.029924  0.000000  0.021515   
2  0.000000  0.00000  0.00000  0.000000  0.000000  0.000000  0.000000   
3  0.000000  0.00000  0.00000  0.000000  0.057354  0.000000  0.041238   
4  0.000000  0.00000  0.00000  0.000000  0.000000  0.000000  0.010758   

     stabil     float     bridg  ...  mayer  importantli  toneprint  \
0  0.139794  0.168192  0.097901  ...    0.0          0.0        0.0   
1  0.000000  0.000000  0.000000  ...    0.0          0.0        0.0   
2  0.000000  0.000000  0.000000  ...    0.0          0.0        0.0   
3  0.000000  0.000000  0.000000  ...    0.0          0.0        0.0   
4  0.000000  0.000000  0.000000  ...    0.0          0.0        0.0   

   stringthru  stopflair  biggi  accord  screenshot  amazoncom  piti  
0         0.0        0.0    0.0     0.0         0.0        0.0 

In [6]:
#  calculate cosine similarity between two review texts

def cosine_similarityW(text1, text2):

    tfidf1 = calculate_tfidf(text1, idf)
    tfidf2 = calculate_tfidf(text2, idf)

    # Calculate the dot product of tfidf1 and tfidf2
    dot_product = sum([tfidf1[word] * tfidf2[word] for word in tfidf1 if word in tfidf2])

    # Calculate the magnitude of tfidf1 and tfidf2
    magnitude1 = math.sqrt(sum([tfidf1[word]**2 for word in tfidf1]))
    magnitude2 = math.sqrt(sum([tfidf2[word]**2 for word in tfidf2]))

    # Calculate the cosine similarity
    similarity = dot_product / (magnitude1 * magnitude2)
    return similarity

#  calculate cosine similarity between two review texts
text1 = data['Review Text'][0]
text2 = data['Review Text'][1]
similarity = cosine_similarity(text1, text2)
print(similarity)

NameError: name 'cosine_similarity' is not defined

In [3]:
#  read data from pickle file
import pickle

# Load the preprocessed data from the pickle file
with open('data.pkl', 'rb') as file:
    data = pickle.load(file)
    
data.head()

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15668\3489083747.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  data = pickle.load(file)


,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,love vintag spring vintag strat good tension g...,"[[[0.35204768, 0.53676, 0.49917382, 0.42068002..."
1,1205,[https://images-na.ssl-images-amazon.com/image...,work great guitar bench mat rug enough abus ta...,"[[[0.35107985, 0.5320194, 0.4778194, 0.4419130..."
2,1708,[https://images-na.ssl-images-amazon.com/image...,use everyth acoust bass ukulel know smaller mo...,"[[[0.38004225, 0.5307589, 0.4896015, 0.4049801..."
3,2078,[https://images-na.ssl-images-amazon.com/image...,great price good qualiti didnt quit match radi...,"[[[0.34581208, 0.55054736, 0.5456345, 0.385915..."
4,801,[https://images-na.ssl-images-amazon.com/image...,bought bass split time primari bass dean edg m...,"[[[0.3345537, 0.50541735, 0.48309487, 0.465147..."


In [9]:
#  creaate a ranked index of the most similar review texts

def most_similar_reviews(query_text, num_results=5):
    query_text = preprocess_review_text(query_text)
    #  calulate the tfidf of the query text
    query_tfidf = calculate_tfidf(query_text, idf)
    #  calculate the similarity of the query text to each review text
    similarities = [sum([query_tfidf[word] * tfidf[word] for word in query_tfidf if word in tfidf]) for tfidf in tfidf_matrix]
    similarities = np.array(similarities)
    

    # Sort the review texts by their similarity to the query text
    most_similar_indices = np.argsort(similarities)[::-1]

    # Return the top N most similar review texts
    return most_similar_indices[:num_results]

#  print the most similar review texts

query_text = "I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break."

similar_reviews = most_similar_reviews(query_text)
print(similar_reviews)

#  print the texts of the most similar review texts
for i in similar_reviews:
    print(data['Review Text'][i])
    print('\n')


NameError: name 'preprocess_review_text' is not defined

In [ ]:
#  find the most similar review to the input text 
query_text = "I love the color and the fabric of this dress. The fit is perfect!"
similar_reviews = most_similar_reviews(query_text)
print(similar_reviews)

#  print the texts of the most similar review texts
for i in similar_reviews:
    print(data['Review Text'][i])
    print('\n')

# 

In [27]:
print(cosine_similarity(data['Review Text'][0], data['Review Text'][271]))
print(cosine_similarity(data['Review Text'][0], data['Review Text'][805]))
print(cosine_similarity(data['Review Text'][0], data['Review Text'][750]))
print(cosine_similarity(data['Review Text'][0], data['Review Text'][758]))    

0.3270878334937966
0.23373038454347247
0.21895440353789145
0.151745632178107


In [28]:
print(data['Review Text'][0])

love vintag spring vintag strat good tension great stabil float bridg want spring way go


In [29]:
#  save the tfidf matrix to a csv file
tfidf_df.to_csv('tfidf.csv', index=False)

# load the tfidf matrix from the csv file
loaded_tfidf_df = pd.read_csv('tfidf.csv')
print(loaded_tfidf_df.head())

       love   vintag   spring     strat      good   tension     great  \
0  0.060206  0.23263  0.28732  0.091102  0.045883  0.139794  0.032990   
1  0.000000  0.00000  0.00000  0.000000  0.029924  0.000000  0.021515   
2  0.000000  0.00000  0.00000  0.000000  0.000000  0.000000  0.000000   
3  0.000000  0.00000  0.00000  0.000000  0.057354  0.000000  0.041238   
4  0.000000  0.00000  0.00000  0.000000  0.000000  0.000000  0.010758   

     stabil     float     bridg  ...  mayer  importantli  toneprint  \
0  0.139794  0.168192  0.097901  ...    0.0          0.0        0.0   
1  0.000000  0.000000  0.000000  ...    0.0          0.0        0.0   
2  0.000000  0.000000  0.000000  ...    0.0          0.0        0.0   
3  0.000000  0.000000  0.000000  ...    0.0          0.0        0.0   
4  0.000000  0.000000  0.000000  ...    0.0          0.0        0.0   

   stringthru  stopflair  biggi  accord  screenshot  amazoncom  piti  
0         0.0        0.0    0.0     0.0         0.0        0.0 

In [30]:
data.head()

,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,love vintag spring vintag strat good tension g...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."
1,1205,[https://images-na.ssl-images-amazon.com/image...,work great guitar bench mat rug enough abus ta...,"[array([[0.38690877, 0.5909363 , 0.47527072, ...."
2,1708,[https://images-na.ssl-images-amazon.com/image...,use everyth acoust bass ukulel know smaller mo...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."
3,2078,[https://images-na.ssl-images-amazon.com/image...,great price good qualiti didnt quit match radi...,"[array([[0.38690877, 0.5909363 , 0.47527072, ...."
4,801,[https://images-na.ssl-images-amazon.com/image...,bought bass split time primari bass dean edg m...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."
